## Convert all data to the same resolution and save the subarea of the gulf of Riga


In [ ]:
!python --version

Python 3.9.16


In [15]:
# import all necessary libraries
import os
from datetime import datetime, timedelta
import getpass
import xarray as xr
!python -m pip install motuclient==1.8.4 --no-cache-dir 
import motuclient
import xarray as xr
import numpy as np
!python -m pip install ffmpeg==1.4
import matplotlib.pyplot as plt
import matplotlib
import subprocess
import pandas as pd
!python -m pip install rioxarray
import rioxarray
import rasterio as rio
import rasterio.plot
!python -m pip install netCDF4
import netCDF4
!python -m pip install h5netcdf
import h5netcdf 
import random
from skimage import io as skio
import ffmpeg
import matplotlib.gridspec as gridspec
import subprocess
from sklearn.neighbors import KernelDensity
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#from numba import jit, cuda
import tifffile
from tqdm import tqdm
import skimage.transform as sktransform
import tensorflow as tf
!pip install dask==2022.12.1
import dask
from dask.graph_manipulation import bind, wait_on

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
chl_dir = r'/content/drive/MyDrive/MasterThesis/Data/CHL_1km'
waveheight_dir1 = r'/content/drive/MyDrive/MasterThesis/Data/Waveheight_old'
waveheight_dir2 = r'/content/drive/MyDrive/MasterThesis/Data/Waveheight_new'
mask_dir = r'/content/drive/MyDrive/MasterThesis/Data/land_mask/iowtopo2_rev03.nc'

data_dir_RIGA = r'/content/drive/MyDrive/MasterThesis/Data_preprocessing/GulfofRiga'
data_dir_KTT = r'/content/drive/MyDrive/MasterThesis/Data_preprocessing/Kattegat'
data_dir_GTL = r'/content/drive/MyDrive/MasterThesis/Data_preprocessing/Gotland'

In [ ]:
riga_range_x  = range(720,880,1)
riga_range_y = range(330,490,1)

kattegat_range_x  = range(50,210,1)
kattegat_range_y = range(200,360,1)

gotland_range_x  = range(470,630,1)
gotland_range_y = range(160,320,1)

## CHL data

In [ ]:
chl_data = xr.open_mfdataset(f"{chl_dir}/CHL_data_"+"*2021.nc", decode_coords="all")
chl_data = chl_data.sel(time=~chl_data.get_index("time").duplicated())
chl_data = chl_data.rename_dims({'lon': 'x', 'lat': 'y'}).rename_vars({"lon": "x", "lat": "y"}).set_coords(["x", "y"])

### Waveheight Data

Load datasets one after the other, split into smaller time chunks and load in txt file for later reading to pd dataframe

In [ ]:
waveheight1_data = xr.open_mfdataset(f"{waveheight_dir1}/WHold_data_"+"*.nc", data_vars='all', coords='minimal')
waveheight1_data

<xarray.Dataset>
Dimensions:  (lon: 755, time: 26983, lat: 757)
Coordinates:
  * lon      (lon) float32 9.264 9.292 9.319 9.347 ... 30.12 30.15 30.18 30.21
  * time     (time) datetime64[ns] 2016-05-01T13:00:00 ... 2019-05-21T13:00:00
  * lat      (lat) float32 53.26 53.27 53.29 53.31 ... 65.81 65.82 65.84 65.86
Data variables:
    VHM0     (time, lat, lon) float32 dask.array<chunksize=(121, 757, 755), meta=np.ndarray>
    VMDR     (time, lat, lon) float32 dask.array<chunksize=(121, 757, 755), meta=np.ndarray>
Attributes: (12/14)
    Conventions:                                CF-1.0
    source:                                     FMI-WAM_CMEMS
    institution:                                Baltic MFC, PU Finnish Meteor...
    title:                                      CMEMS WAM hindcast (hourly)
    FROM_ORIGINAL_FILE__westernmost_longitude:  9.0138
    FROM_ORIGINAL_FILE__easternmost_longitude:  30.2080
    ...                                         ...
    grid_resolution:                            1 nautical mile (ie. 0.0167 d...
    cmems_product_id:                           BALTICSEA_REANALYSIS_WAV_003_015
    _CoordSysBuilder:                           ucar.nc2.dataset.conv.CF1Conv...
    references:                                  
    comment:                                    
    history:                                    Data extracted from dataset h...

In [16]:
time_value = waveheight1_data.VHM0.shape[0]
print(time_value)

26983


In [25]:
lat_lon_subset = chl_data[['x', 'y']]
old_end_time = 0
VHM0_1_RIGA = np.empty((0,160,160))
VMDR_1_RIGA = np.empty((0,160,160))

VHM0_1_KTT = np.empty((0,160,160))
VMDR_1_KTT = np.empty((0,160,160))

VHM0_1_GTL = np.empty((0,160,160))
VMDR_1_GTL = np.empty((0,160,160))
for i in tqdm(range(0,time_value,1000)): #26983
  waveheight1_data_ = waveheight1_data.rename_dims({'lon': 'x', 'lat': 'y'}).rename_vars({"lon": "x", "lat": "y"}).set_coords(["x", "y"])
  
  if i+1000>time_value:
    waveheight1_data_ = waveheight1_data_.isel(time=range(i,time_value,1))
    print(waveheight1_data_.time[time_value-i-1].dt.date.data)
    expected_times = pd.period_range(start=str(waveheight1_data_.time[0].dt.date.data), end=str(waveheight1_data_.time[time_value-i-1].dt.date.data), freq='D')
  else:
    waveheight1_data_ = waveheight1_data_.isel(time=range(i,i+1000,1))
    print(waveheight1_data_.time[999].dt.date.data)
    expected_times = pd.period_range(start=str(waveheight1_data_.time[0].dt.date.data), end=str(waveheight1_data_.time[999].dt.date.data), freq='D')
    print(expected_times)
  
  waveheight1_data_ = waveheight1_data_.resample(time='1D').mean('time')
  
  #waveheight1_data_ = waveheight1_data_.reindex(time=expected_times)
  #waveheight1_data_ = waveheight1_data_.compute()

  waveheight1_data_resampled = waveheight1_data_.interp_like(lat_lon_subset, method='linear').compute()
  print(np.count_nonzero(~np.isnan(waveheight1_data_['VMDR'])))

  waveheight1_data_RIGA = waveheight1_data_resampled.isel(x=riga_range_x, y= riga_range_y)
  waveheight1_data_KTT = waveheight1_data_resampled.isel(x=kattegat_range_x, y= kattegat_range_y)
  waveheight1_data_GTL = waveheight1_data_resampled.isel(x=gotland_range_x, y= gotland_range_y)
  
  VHM0_RIGA = waveheight1_data_RIGA['VHM0'].values
  print(VHM0_RIGA.shape)
  VMDR_RIGA = waveheight1_data_RIGA['VMDR'].values
  print(np.count_nonzero(~np.isnan(VMDR_RIGA)))

  VHM0_KTT = waveheight1_data_KTT['VHM0'].values
  print(VHM0_KTT.shape)
  VMDR_KTT = waveheight1_data_KTT['VMDR'].values
  VHM0_GTL = waveheight1_data_GTL['VHM0'].values
  print(VHM0_GTL.shape)
  VMDR_GTL = waveheight1_data_GTL['VMDR'].values
  if old_end_time != expected_times[0]:
    VHM0_1_RIGA = np.append(VHM0_1_RIGA, VHM0_RIGA[:1,:,:], axis=0)
    VMDR_1_RIGA = np.append(VMDR_1_RIGA, VMDR_RIGA[:1,:,:], axis=0)
    VHM0_1_KTT = np.append(VHM0_1_KTT, VHM0_KTT[:1,:,:], axis=0)
    VMDR_1_KTT = np.append(VMDR_1_KTT, VMDR_KTT[:1,:,:], axis=0)
    VHM0_1_GTL = np.append(VHM0_1_GTL, VHM0_GTL[:1,:,:], axis=0)
    VMDR_1_GTL = np.append(VMDR_1_GTL, VMDR_GTL[:1,:,:], axis=0)
    
  else:
    VHM0_1_RIGA[-1,:,:] = (VHM0_1_RIGA[-1,:,:] + VHM0_RIGA[0,:,:])/2
    VMDR_1_RIGA[-1,:,:] = (VMDR_1_RIGA[-1,:,:] + VMDR_RIGA[0,:,:])/2
    VHM0_1_KTT[-1,:,:] = (VHM0_1_KTT[-1,:,:] + VHM0_KTT[0,:,:])/2
    VMDR_1_KTT[-1,:,:] = (VMDR_1_KTT[-1,:,:] + VMDR_KTT[0,:,:])/2
    VHM0_1_GTL[-1,:,:] = (VHM0_1_GTL[-1,:,:] + VHM0_GTL[0,:,:])/2
    VMDR_1_GTL[-1,:,:] = (VMDR_1_GTL[-1,:,:] + VMDR_GTL[0,:,:])/2

  VHM0_1_RIGA = np.append(VHM0_1_RIGA, VHM0_RIGA[1:,:,:], axis=0)
  VMDR_1_RIGA = np.append(VMDR_1_RIGA, VMDR_RIGA[1:,:,:], axis=0)
  VHM0_1_KTT = np.append(VHM0_1_KTT, VHM0_KTT[1:,:,:], axis=0)
  VMDR_1_KTT = np.append(VMDR_1_KTT, VMDR_KTT[1:,:,:], axis=0)
  VHM0_1_GTL = np.append(VHM0_1_GTL, VHM0_GTL[1:,:,:], axis=0)
  VMDR_1_GTL = np.append(VMDR_1_GTL, VMDR_GTL[1:,:,:], axis=0)
  print(VHM0_1_RIGA.shape)
  print(VHM0_1_KTT.shape)
  print(VHM0_1_GTL.shape)
  old_end_time = expected_times[-1]



  0%|          | 0/27 [00:00<?, ?it/s]

2016-06-11
PeriodIndex(['2016-05-01', '2016-05-02', '2016-05-03', '2016-05-04',
             '2016-05-05', '2016-05-06', '2016-05-07', '2016-05-08',
             '2016-05-09', '2016-05-10', '2016-05-11', '2016-05-12',
             '2016-05-13', '2016-05-14', '2016-05-15', '2016-05-16',
             '2016-05-17', '2016-05-18', '2016-05-19', '2016-05-20',
             '2016-05-21', '2016-05-22', '2016-05-23', '2016-05-24',
             '2016-05-25', '2016-05-26', '2016-05-27', '2016-05-28',
             '2016-05-29', '2016-05-30', '2016-05-31', '2016-06-01',
             '2016-06-02', '2016-06-03', '2016-06-04', '2016-06-05',
             '2016-06-06', '2016-06-07', '2016-06-08', '2016-06-09',
             '2016-06-10', '2016-06-11'],
            dtype='period[D]')


  4%|▎         | 1/27 [01:24<36:41, 84.68s/it]

6130281
(42, 160, 160)
513492
(42, 160, 160)
(42, 160, 160)
(42, 160, 160)
(42, 160, 160)
(42, 160, 160)
2016-07-23
PeriodIndex(['2016-06-11', '2016-06-12', '2016-06-13', '2016-06-14',
             '2016-06-15', '2016-06-16', '2016-06-17', '2016-06-18',
             '2016-06-19', '2016-06-20', '2016-06-21', '2016-06-22',
             '2016-06-23', '2016-06-24', '2016-06-25', '2016-06-26',
             '2016-06-27', '2016-06-28', '2016-06-29', '2016-06-30',
             '2016-07-01', '2016-07-02', '2016-07-03', '2016-07-04',
             '2016-07-05', '2016-07-06', '2016-07-07', '2016-07-08',
             '2016-07-09', '2016-07-10', '2016-07-11', '2016-07-12',
             '2016-07-13', '2016-07-14', '2016-07-15', '2016-07-16',
             '2016-07-17', '2016-07-18', '2016-07-19', '2016-07-20',
             '2016-07-21', '2016-07-22', '2016-07-23'],
            dtype='period[D]')


  7%|▋         | 2/27 [02:43<33:44, 80.98s/it]

...

(1034, 160, 160)
(1034, 160, 160)
(1034, 160, 160)
2019-04-10
PeriodIndex(['2019-02-28', '2019-03-01', '2019-03-02', '2019-03-03',
             '2019-03-04', '2019-03-05', '2019-03-06', '2019-03-07',
             '2019-03-08', '2019-03-09', '2019-03-10', '2019-03-11',
             '2019-03-12', '2019-03-13', '2019-03-14', '2019-03-15',
             '2019-03-16', '2019-03-17', '2019-03-18', '2019-03-19',
             '2019-03-20', '2019-03-21', '2019-03-22', '2019-03-23',
             '2019-03-24', '2019-03-25', '2019-03-26', '2019-03-27',
             '2019-03-28', '2019-03-29', '2019-03-30', '2019-03-31',
             '2019-04-01', '2019-04-02', '2019-04-03', '2019-04-04',
             '2019-04-05', '2019-04-06', '2019-04-07', '2019-04-08',
             '2019-04-09', '2019-04-10'],
            dtype='period[D]')
5464501
(42, 160, 160)
495413
(42, 160, 160)
(42, 160, 160)


 96%|█████████▋| 26/27 [42:27<01:48, 108.25s/it]

(1075, 160, 160)
(1075, 160, 160)
(1075, 160, 160)
2019-05-21
5994353
(42, 160, 160)
513492
(42, 160, 160)
(42, 160, 160)


100%|██████████| 27/27 [44:04<00:00, 97.96s/it] 

(1116, 160, 160)
(1116, 160, 160)
(1116, 160, 160)


In [26]:
np.count_nonzero(~np.isnan(VMDR_1_RIGA))

13027925

In [27]:
print(len(pd.period_range(start="2016-05-01",end="2019-05-21")))

1116


In [28]:
VHM0_1_RIGA.shape

(1116, 160, 160)

In [29]:
np.save(f"{data_dir_RIGA}/VHM01_data.npy", VHM0_1_RIGA)
np.save(f"{data_dir_RIGA}/VMDR1_data.npy", VMDR_1_RIGA)
np.save(f"{data_dir_KTT}/VHM01_data.npy", VHM0_1_KTT)
np.save(f"{data_dir_KTT}/VMDR1_data.npy", VMDR_1_KTT)
np.save(f"{data_dir_GTL}/VHM01_data.npy", VHM0_1_GTL)
np.save(f"{data_dir_GTL}/VMDR1_data.npy", VMDR_1_GTL)

Second Dataset

In [ ]:
waveheight2_data = xr.open_mfdataset(f"{waveheight_dir2}/WH_new_data_"+"*.nc", data_vars='all', coords='minimal')
waveheight2_data

<xarray.Dataset>
Dimensions:  (lon: 755, time: 34848, lat: 757)
Coordinates:
  * lon      (lon) float32 9.264 9.292 9.319 9.347 ... 30.12 30.15 30.18 30.21
  * time     (time) datetime64[ns] 2019-05-16 2019-05-16T01:00:00 ... 2023-04-26
  * lat      (lat) float32 53.26 53.27 53.29 53.31 ... 65.81 65.82 65.84 65.86
Data variables:
    VHM0     (time, lat, lon) float32 dask.array<chunksize=(121, 757, 755), meta=np.ndarray>
    VMDR     (time, lat, lon) float32 dask.array<chunksize=(121, 757, 755), meta=np.ndarray>
Attributes: (12/14)
    Conventions:                                CF-1.0
    source:                                     FMI-WAM_CMEMS
    institution:                                Baltic MFC, PU Finnish Meteor...
    title:                                      CMEMS WAM model fields (hourly)
    cmems_product_id:                           BALTICSEA_ANALYSISFORECAST_WA...
    FROM_ORIGINAL_FILE__westernmost_longitude:  9.0138
    ...                                         ...
    FROM_ORIGINAL_FILE__northernmost_latitude:  65.9081
    grid_resolution:                            1 nautical mile (ie. 0.0167 d...
    _CoordSysBuilder:                           ucar.nc2.dataset.conv.CF1Conv...
    references:                                  
    comment:                                    
    history:                                    Data extracted from dataset h...

In [ ]:
waveheight2_data = waveheight2_data.sel(time=slice('2019-05-22', '2023-04-14'))

In [30]:
time_value = waveheight2_data.VHM0.shape[0]
print(time_value)

34436


In [31]:
lat_lon_subset = chl_data[['x', 'y']]
j = 0
old_end_time = 0
VHM0_2_RIGA = np.empty((0,160,160))
VMDR_2_RIGA = np.empty((0,160,160))

VHM0_2_KTT = np.empty((0,160,160))
VMDR_2_KTT = np.empty((0,160,160))

VHM0_2_GTL = np.empty((0,160,160))
VMDR_2_GTL = np.empty((0,160,160))
for i in tqdm(range(0,time_value,1000)): #26983
  waveheight1_data_ = waveheight2_data.rename_dims({'lon': 'x', 'lat': 'y'}).rename_vars({"lon": "x", "lat": "y"}).set_coords(["x", "y"])
  
  if i+1000>time_value:
    waveheight1_data_ = waveheight1_data_.isel(time=range(i,time_value,1))
    print(waveheight1_data_.time[time_value-i-1].dt.date.data)
    expected_times = pd.period_range(start=str(waveheight1_data_.time[0].dt.date.data), end=str(waveheight1_data_.time[time_value-i-1].dt.date.data), freq='D')
    print(expected_times)
  else:
    waveheight1_data_ = waveheight1_data_.isel(time=range(i,i+1000,1))
    print(waveheight1_data_.time[999].dt.date.data)
    expected_times = pd.period_range(start=str(waveheight1_data_.time[0].dt.date.data), end=str(waveheight1_data_.time[999].dt.date.data), freq='D')
    print(expected_times)
  waveheight1_data_ = waveheight1_data_.resample(time='1D').mean('time')

  
  #waveheight1_data_ = waveheight1_data_.reindex(time=expected_times)
  #waveheight1_data_ = waveheight1_data_.compute()

  waveheight1_data_resampled = waveheight1_data_.interp_like(lat_lon_subset, method='linear').compute()

  waveheight1_data_RIGA = waveheight1_data_resampled.isel(x=riga_range_x, y= riga_range_y)
  waveheight1_data_KTT = waveheight1_data_resampled.isel(x=kattegat_range_x, y= kattegat_range_y)
  waveheight1_data_GTL = waveheight1_data_resampled.isel(x=gotland_range_x, y= gotland_range_y)
  
  VHM0_RIGA = waveheight1_data_RIGA['VHM0'].values
  print(VHM0_RIGA.shape)
  VMDR_RIGA = waveheight1_data_RIGA['VMDR'].values

  VHM0_KTT = waveheight1_data_KTT['VHM0'].values
  print(VHM0_KTT.shape)
  VMDR_KTT = waveheight1_data_KTT['VMDR'].values
  VHM0_GTL = waveheight1_data_GTL['VHM0'].values
  print(VHM0_GTL.shape)
  VMDR_GTL = waveheight1_data_GTL['VMDR'].values
  if old_end_time != expected_times[0]:
    VHM0_2_RIGA = np.append(VHM0_2_RIGA, VHM0_RIGA[:1,:,:], axis=0)
    VMDR_2_RIGA = np.append(VMDR_2_RIGA, VMDR_RIGA[:1,:,:], axis=0)
    VHM0_2_KTT = np.append(VHM0_2_KTT, VHM0_KTT[:1,:,:], axis=0)
    VMDR_2_KTT = np.append(VMDR_2_KTT, VMDR_KTT[:1,:,:], axis=0)
    VHM0_2_GTL = np.append(VHM0_2_GTL, VHM0_GTL[:1,:,:], axis=0)
    VMDR_2_GTL = np.append(VMDR_2_GTL, VMDR_GTL[:1,:,:], axis=0)
    
  else:
    VHM0_2_RIGA[-1,:,:] = (VHM0_2_RIGA[-1,:,:] + VHM0_RIGA[0,:,:])/2
    VMDR_2_RIGA[-1,:,:] = (VMDR_2_RIGA[-1,:,:] + VMDR_RIGA[0,:,:])/2
    VHM0_2_KTT[-1,:,:] = (VHM0_2_KTT[-1,:,:] + VHM0_KTT[0,:,:])/2
    VMDR_2_KTT[-1,:,:] = (VMDR_2_KTT[-1,:,:] + VMDR_KTT[0,:,:])/2
    VHM0_2_GTL[-1,:,:] = (VHM0_2_GTL[-1,:,:] + VHM0_GTL[0,:,:])/2
    VMDR_2_GTL[-1,:,:] = (VMDR_2_GTL[-1,:,:] + VMDR_GTL[0,:,:])/2

  VHM0_2_RIGA = np.append(VHM0_2_RIGA, VHM0_RIGA[1:,:,:], axis=0)
  VMDR_2_RIGA = np.append(VMDR_2_RIGA, VMDR_RIGA[1:,:,:], axis=0)
  VHM0_2_KTT = np.append(VHM0_2_KTT, VHM0_KTT[1:,:,:], axis=0)
  VMDR_2_KTT = np.append(VMDR_2_KTT, VMDR_KTT[1:,:,:], axis=0)
  VHM0_2_GTL = np.append(VHM0_2_GTL, VHM0_GTL[1:,:,:], axis=0)
  VMDR_2_GTL = np.append(VMDR_2_GTL, VMDR_GTL[1:,:,:], axis=0)
  print(VHM0_2_RIGA.shape)
  print(VHM0_2_KTT.shape)
  print(VHM0_2_GTL.shape)
  old_end_time = expected_times[-1]

  0%|          | 0/35 [00:00<?, ?it/s]

2019-07-02
PeriodIndex(['2019-05-22', '2019-05-23', '2019-05-24', '2019-05-25',
             '2019-05-26', '2019-05-27', '2019-05-28', '2019-05-29',
             '2019-05-30', '2019-05-31', '2019-06-01', '2019-06-02',
             '2019-06-03', '2019-06-04', '2019-06-05', '2019-06-06',
             '2019-06-07', '2019-06-08', '2019-06-09', '2019-06-10',
             '2019-06-11', '2019-06-12', '2019-06-13', '2019-06-14',
             '2019-06-15', '2019-06-16', '2019-06-17', '2019-06-18',
             '2019-06-19', '2019-06-20', '2019-06-21', '2019-06-22',
             '2019-06-23', '2019-06-24', '2019-06-25', '2019-06-26',
             '2019-06-27', '2019-06-28', '2019-06-29', '2019-06-30',
             '2019-07-01', '2019-07-02'],
            dtype='period[D]')


  3%|▎         | 1/35 [01:20<45:31, 80.35s/it]

(42, 160, 160)
(42, 160, 160)
(42, 160, 160)
(42, 160, 160)
(42, 160, 160)
(42, 160, 160)
2019-08-12
PeriodIndex(['2019-07-02', '2019-07-03', '2019-07-04', '2019-07-05',
             '2019-07-06', '2019-07-07', '2019-07-08', '2019-07-09',
             '2019-07-10', '2019-07-11', '2019-07-12', '2019-07-13',
             '2019-07-14', '2019-07-15', '2019-07-16', '2019-07-17',
             '2019-07-18', '2019-07-19', '2019-07-20', '2019-07-21',
             '2019-07-22', '2019-07-23', '2019-07-24', '2019-07-25',
             '2019-07-26', '2019-07-27', '2019-07-28', '2019-07-29',
             '2019-07-30', '2019-07-31', '2019-08-01', '2019-08-02',
             '2019-08-03', '2019-08-04', '2019-08-05', '2019-08-06',
             '2019-08-07', '2019-08-08', '2019-08-09', '2019-08-10',
             '2019-08-11', '2019-08-12'],
            dtype='period[D]')


  6%|▌         | 2/35 [02:30<40:47, 74.17s/it]

...

(1365, 160, 160)
(1365, 160, 160)
(1365, 160, 160)
2023-03-27
PeriodIndex(['2023-02-14', '2023-02-15', '2023-02-16', '2023-02-17',
             '2023-02-18', '2023-02-19', '2023-02-20', '2023-02-21',
             '2023-02-22', '2023-02-23', '2023-02-24', '2023-02-25',
             '2023-02-26', '2023-02-27', '2023-02-28', '2023-03-01',
             '2023-03-02', '2023-03-03', '2023-03-04', '2023-03-05',
             '2023-03-06', '2023-03-07', '2023-03-08', '2023-03-09',
             '2023-03-10', '2023-03-11', '2023-03-12', '2023-03-13',
             '2023-03-14', '2023-03-15', '2023-03-16', '2023-03-17',
             '2023-03-18', '2023-03-19', '2023-03-20', '2023-03-21',
             '2023-03-22', '2023-03-23', '2023-03-24', '2023-03-25',
             '2023-03-26', '2023-03-27'],
            dtype='period[D]')
(42, 160, 160)
(42, 160, 160)
(42, 160, 160)


 97%|█████████▋| 34/35 [48:45<01:33, 93.22s/it]

(1406, 160, 160)
(1406, 160, 160)
(1406, 160, 160)
2023-04-14
PeriodIndex(['2023-03-27', '2023-03-28', '2023-03-29', '2023-03-30',
             '2023-03-31', '2023-04-01', '2023-04-02', '2023-04-03',
             '2023-04-04', '2023-04-05', '2023-04-06', '2023-04-07',
             '2023-04-08', '2023-04-09', '2023-04-10', '2023-04-11',
             '2023-04-12', '2023-04-13', '2023-04-14'],
            dtype='period[D]')
(19, 160, 160)
(19, 160, 160)
(19, 160, 160)


100%|██████████| 35/35 [49:19<00:00, 84.55s/it]

(1424, 160, 160)
(1424, 160, 160)
(1424, 160, 160)


In [32]:
np.save(f"{data_dir_RIGA}/VHM02_data.npy", VHM0_2_RIGA)
np.save(f"{data_dir_RIGA}/VMDR2_data.npy", VMDR_2_RIGA)
np.save(f"{data_dir_KTT}/VHM02_data.npy", VHM0_2_KTT)
np.save(f"{data_dir_KTT}/VMDR2_data.npy", VMDR_2_KTT)
np.save(f"{data_dir_GTL}/VHM02_data.npy", VHM0_2_GTL)
np.save(f"{data_dir_GTL}/VMDR2_data.npy", VMDR_2_GTL)

In [34]:
print(np.count_nonzero(~np.isnan(VMDR_2_RIGA)))
print(len(pd.period_range(start="2019-05-22",end="2023-04-14")))
VHM0_2_RIGA.shape

17285759
1424


(1424, 160, 160)